# 01 - Objects and Classes

We will try to refer to instances of classes as **instances** and not **objects** throughout this course because pretty much everything is an object.

The class itself is also called a `type`. For example, we call `list` a type but it's also a `list` class.

The type of our classes (not instances) are of type: `type`. This includes things like the `list` class - we'll look into all of this later.

The type of `type` is also of type: `type`. Again, we'll look into all of this later.

# 02 - Class Attributes

Everything in this subsection is related to the class itself, **not the instances of the class**.

We can **get** the attributes of **any object** in general, not just classes, using the `getattr` function:
```python
getattr(object_symbol, attribute_name, optional_default)
```
- The shorthand dot notation is identical to this except you can't specify defaults with dot notation.
- When bounded to a class, they are often called **data attributes** or **class attributes** in contrast to **instance attributes**.

Here's a simple example:

In [14]:
class MyClass:
    language = 'Python'
    version = '3.6'

print(getattr(MyClass, 'language'))
print(getattr(MyClass, 'x', 'N/A'))

Python
N/A


We can **set** the attributes of **any object** in general, not just classes, using the `setattr` function:
```python
getattr(object_symbol, attribute_name, attribute value)
```
- Since python is a dynamic language, we can define attributes in our code to modify our classes at runtime.

Here's a simple example:

In [15]:
class MyClass:
    language = 'Python'
    version = '3.6'

setattr(MyClass, 'version', '3.7')
print(getattr(MyClass, 'version'))

setattr(MyClass, 'x', 100)
print(getattr(MyClass, 'x'))

3.7
100


We can **delete** the attributes of **any object** (usually), not just classes, using the `delattr` function:
```python
delattr(object_symbol, attribute_name)
```
- We can also use the `del` keyword which we've seen in the past for removing keys from dictionaries. This fact will become more relevant after reading down below.

**Where is the state stored?**

- The state of a class is stored in a dictionary accessed with `__dict__`.
- This returns a **mappingproxy** which is a hashmap (dictionary) but not a python `dict`. It is essentially a read-only dictionary that's only mutable via `setattr`. This is in contrast to the state of an instance accessed by `__dict__` which *is* a python `dict` and is therefore read-and-write.
- This dictionary is often called the **class namespace**.

In [16]:
MyClass.__dict__

mappingproxy({'__module__': '__main__',
              'language': 'Python',
              'version': '3.7',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None,
              'x': 100})

While we can get (but not set) attributes via this dictionary, it's better to use `getattr` or the dot notation as classes can sometimes have attributes that **don't** show up in this dictionary e.g. `__name__`.

In [17]:
MyClass.__dict__['language']

'Python'

# 03 - Callable Class Attributes

Class attributes can be any object type, including callables such as functions. These are **not** instance methods as they do not take `self` in their argument. They are typically called **function attributes**.

In [19]:
class Program:
    language = 'Python'
    
    def say_hello():
        print(f'Hello from {Program.language}!')

Since this function is an **attribute**, we can use `getattr` or dot notation:

In [21]:
Program.say_hello()
getattr(Program, 'say_hello')()

Hello from Python!
Hello from Python!


# 04 - Classes are Callable

When we use the `class` keyword, Python automatically adds behaviours to the class.

It makes it a callable with a return value that is an object which we call an **instance**.

The **instance** has its own **_distinct_** namespace which is separate to the namespace of the class itself.

`__dict__` on the instances returns the instance's **local** namespace while `__dict__` on the class returns the class' namespace.

In [28]:
class Program:
    language = 'Python'
    
    def say_hello():
        print(f'Hello from {Program.language}!')

In [29]:
Program.__dict__

mappingproxy({'__module__': '__main__',
              'language': 'Python',
              'say_hello': <function __main__.Program.say_hello()>,
              '__dict__': <attribute '__dict__' of 'Program' objects>,
              '__weakref__': <attribute '__weakref__' of 'Program' objects>,
              '__doc__': None})

In [30]:
p.__dict__

{}

We should **always** use the `type` function over the `__class__` attribute to access the original class.

This is because `__class__` is just an attribute that can be manually modified:

In [34]:
class MyClass:
    __class__ = str

m = MyClass()

In [35]:
print(m.__class__)
print(type(m))

<class 'str'>
<class '__main__.MyClass'>


The `isinstance` function *is* affected by our modification too. This is more related to **metaprogramming** so we'll cover it in more detail later, but to demonstrate:

In [39]:
print(isinstance(m, str))
print(isinstance(m, MyClass))

True
True


But we should definitely use **isinstance** when it comes to inheritance which we'll also look at later.

# 05 - Data Attributes

Let's illustrate the differences between the class and instance namespaces with an example. 

We will demonstrate how performing a lookup for an instance attribute will look in the local namespace, and if it doesn't find it there, it will look in the class namespace:

In [47]:
class BankAccount:
    apr = 1.2

In [52]:
acc_1 = BankAccount()
acc_2 = BankAccount()

acc_1.__dict__

{}

In [50]:
acc_1.apr

1.2

If we set the **instance attribute**, only that instance's namespace will be affected. Other instance namespaces and the class namespace itself will be unaffected.

In [56]:
acc_1.apr = 0

acc_1.__dict__, acc_2.__dict__

({'apr': 0}, {})

If we get the **instance attribute** back, we'll always search in the local namespace first before moving up the inheritance tree:

In [57]:
acc_1.apr, acc_2.apr

(0, 1.2)

# 06 - Function Attributes

So far, we have been dealing with non-callable attributes. When attributes are actually functions, things behave differently.

Let's look at the difference between viewing this function through the class vs. through an instance:

In [5]:
class Person:
    def say_hello():
        print('Hello!')

In [6]:
Person.say_hello

<function __main__.Person.say_hello()>

In [17]:
p = Person()
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x00000229C2158A60>>

In [27]:
type(Person.say_hello), type(p.say_hello)

(function, method)

**Bound methods** 

- These are different from functions.
- They can only be called bound methods once an instance has been made which binds it to the method - otherwise, they're just functions.
- The object that they are bound to is passed (injected) to the method as its **first parameter**.
- Methods are objects and like any objects in Python, it has attributes:
    - `__self__`: the instance the method is bound to.
    - `__func__`: the original function defined in the class. This is how the method knows what function to call.

We cannot call `say_hello` as a bound method since the function hasn't been set up to deal with taking the instance as its first argument. We'll fix that:

In [23]:
class Person:
    language = 'Python'
    def say_hello(obj, name):
        print(f'Hello! {name}! I am {obj.language}.')

As can be seen below, calling the bound method is identical to calling the function with the instance. The `obj.language` used was not found in the local namespace so it was searched for in the class namespace.

By convention, we should use `self` for the name of the instance which we shall do from now on.

In [25]:
python = Person()
python.say_hello('John')
Person.say_hello(python, 'John')

Hello! John! I am Python.
Hello! John! I am Python.


We can add methods *after* creating the class definition (monkeypatching). We must remember to add `self` as the first argument to ensure it's treated like a method:

In [51]:
Person.say_goodbye = lambda self, name: print(f'Goodbye {name}! From {self.language}.')
python.say_goodbye

<bound method <lambda> of <__main__.Person object at 0x00000229C2007A00>>

In [52]:
python.say_goodbye('John')

Goodbye John! From Python.


This runtime initialisation **only affects the bound instance**. If a new instance is created, it will not have access to this method, because `say_goodbye` is only bound to `python`.

In [57]:
java = Person()
java.say_goodbye()

AttributeError: 'Person' object has no attribute 'say_goodbye'

# 07 - Initializing Class Instances

We've seen that when we instantiate a class, we
1. create a **new instance** of the class
2. **initialise** the namespace of the instance: `obj.__dict__ -> {}`

We can "intercept" both the creating and initialization phases, by using special methods `__new__` and `__init__`.

We'll come back to `__new__` later. For now we'll focus on `__init__`.

But as you already might know, we can perform our own initialisation. We do this with `__init__` function which is a **class attribute** in the **class namespace**. It is only when we've created the instance and created the instance namespace (`obj.__dict__ -> {}`) that the `__init__` function is called as a bound method.

In [44]:
class MyClass:
    def __init__(self, version):
        self.version = version

In [46]:
m = MyClass(3.7)
m.version

3.7

In [49]:
MyClass.__init__(m, 3.8)
m.version

3.8

# 08 - Creating Attributes at Run-Time

Another way we can create and bind a method to an instance at runtime is using `MethodType` from the `types` library:
```python
MethodType(function, object)
```
where the `function` is the function we want to bind and the `object` is that object we want to bind it to.

In [59]:
from types import MethodType

class Person:
    language = 'Python'

p = Person()
p.say_hello = MethodType(lambda self, name: f'Hello {name}! I am {self.language}.', p)

In [60]:
p.say_hello('John')

'Hello John! I am Python.'

As seen before, this runtime initialisation **only affects the bound instance**. If a new instance is created, it will not have access to this method, because `say_hello` is only bound to `python`.

In [63]:
java = Person()
java.say_hello('John')

AttributeError: 'Person' object has no attribute 'say_hello'

With this approach, we can define different functions and register them to different instances. The only common thing we would need among the different instances is a register manager written in the original class which would handle the assignment/binding and maybe some error handling. 

See original notebook for an example.

# 09 - Properties

# 10 - Property Decorators

# 11 - Read-Only and Computed Properties

# 12 - Deleting Properties

# 13 - Class and Static Methods

# 14 - Class Body Scope